In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
#%matplotlib  inline
import csv
from Plot_Library1 import *

In [ ]:
#makes it look nice
format_and_print = lambda nums: print(' '.join([f'{num:.2e}' for num in nums])) #[1=>many items are printed]

In [ ]:
def poly_AKS(t,y):
    kd = 2.5e-4
    kp = 1.3e+3
    ktrm = 2.5e-2
    ktc = 3.7e+7
    ktd = 7.3e+7
    f = 1.0 #for comparison
    #f = 0.58 #see if any difference
    
    I , M, lam0,lam1,lam2,mu0,mu1,mu2 = y
   #=============================================  
     #======================================================
    if(mu0 < 0):
        mu0 = 0
    if(mu1 < 0):
        print('mu 1 <0 at t=', mu1,t) #happened at t= 70 at least
        mu1 = 0
    if(mu2 < 0):
        mu2 = 0
    if(lam0 < 0):
         lam0 = 0
    if(lam1 < 0):
         print('lam 1 <0 at t=', mu1,t)
         lam1 = 0
    if(lam2 < 0):
         lam2 = 0    
    #=============================================  
    dIdt = -kd*I
    dMdt = -mu0*M*(kp+ktrm) - 2.*f*kd*I #AKS
    #dead moments
    dlam0dt = 2*f*kd*I + mu0 *ktrm*M - .5*mu0*mu0*ktc
    dlam1dt = 2*f*kd*I + mu0 *(kp+ktrm)*M
    dlam2dt = 2*f*kd*I + mu0 *(kp+ktrm)*M + 2*mu1*kp*M +mu1*mu1*ktc
    #live moments
    dmu0dt =  2*f*kd*I - (mu0 * mu0 * (ktc+ktd) )
    dmu1dt =  2*f*kd*I - (mu1-mu0)*ktrm*M - mu1 * mu0 *(ktc+ktd) + mu0*kp*M
    dmu2dt =  2*f*kd*I - (mu2-mu0)*ktrm*M - mu2 * mu0 *(ktc+ktd) + (2 * mu1 + mu0)*kp*M
    return [dIdt, dMdt, dlam0dt, dlam1dt, dlam2dt, dmu0dt, dmu1dt, dmu2dt ]




In [ ]:
def poly_Dhooge(t,y):
    kd = 2.5e-4
    kp = 1.3e+3
    ktrm = 2.5e-2
    ktc = 3.7e+7
    ktd = 7.3e+7
    f = 1.0,
    
    I2 , M, lam0,lam1,lam2,mu0,mu1,mu2, I = y
     #======================================================
    if(mu0 < 0):
        mu0 = 0
    if(mu1 < 0):
       # print('mu 1 <0 at t=', mu1,t) #happened at t= 70 at least
        mu1 = 0
    if(mu2 < 0):
        mu2 = 0
    if(lam0 < 0):
         lam0 = 0
    if(lam1 < 0):
         #print('lam 1 <0 at t=', lam1,t)
         lam1 = 0
    if(lam2 < 0):
         lam2 = 0    
    #=============================================  
    dI2dt = -kd*I2   
    dIdt = 2*kd*I2- kp*I*M
    #lam0 == [R]tot
    dMdt = -lam0*M*(kp+ktrm) - kp*I*M
    #dead moments in AKS; Dooge equates lambda with macroradicals and mu for Dead polymer
    dmu0dt = .5*lam0*lam0*ktc + lam0*lam0*ktd + ktrm*lam0*M
    #dmu1dt = lam0*lam1*ktc    + lam0*lam1*ktd + lam1*ktrm*M     #I THINKTHIS IS WRONG
    dmu1dt = lam0*lam1*ktc    + lam0*lam1*ktd + lam1*ktrm*M  
    dmu2dt = lam1*lam1*ktc    + lam0*lam2*ktc + lam0*lam2*ktd + ktrm*lam2*M
    #live moments
    dlam0dt =  - (lam0 * lam0 * (ktc+ktd)) + kp*I*M #lamo or 2*f*kd*I
    dlam1dt =  - (lam0 * lam1 * (ktc+ktd)) + (lam0-lam1)*ktrm*M  + lam0*kp*M
    dlam2dt =  - (lam0 * lam2 * (ktc+ktd)) + (lam0-lam2)*ktrm*M  + (2 * lam1 + lam0)*kp*M + kp*I*M
    #rewrite in exact form of paper to determine difference
    #dlam2dt = -ktc*lam0*lam2 -ktd*lam0*lam2 + kp*I*M - ktrm*lam0*M -  ktrm*lam2*M + kp*lam0*M + 2*kp*lam1*M #same result
    return [dI2dt, dMdt, dlam0dt, dlam1dt, dlam2dt, dmu0dt, dmu1dt, dmu2dt, dIdt ]

In [ ]:

M0 = 3.0
S0 =3.0
I0 = M0/100
I20 = I0
V0 = 1
T = 80
lambda0= 2e-9  #initialize living radicals 
lambda1= 2e-9
lambda2= 2e-9
mu0=  2e-9     #initialize dead radicals
mu1=  2e-9
mu2 = 2e-9

y0 = [I0, M0, lambda0, lambda1, lambda2, mu0, mu1, mu2] 
t0 = 0
abserr = 1.e-10 
relerr = 1.e-10
tf = 3000
#force evaluation at every second
t_pts = np.linspace(t0, tf, tf)  # set number of points between t=t0 and t=tf. OPTIONAL
#my preferred starting configuration
sol_AKS =  solve_ivp(poly_AKS, (t0, tf), y0, t_eval=t_pts,   rtol=relerr, atol=abserr, method='BDF', dense_output=True )  


In [ ]:
M0 = 3.0
S0 =3.0
I20 = M0/100
V0 = 1
I0 = 0
T = 80
y0 = [I20, M0, lambda0, lambda1, lambda2, mu0, mu1, mu2, I0] 
lambda0= 2e-9  #initialize living radicals 

lambda1= 2e-9
lambda2= 2e-9
mu0=  2e-9     #initialize dead radicals
mu1=  2e-9
mu2 = 2e-9
I0 = 0
t0 = 0
abserr = 1.e-10 
relerr = 1.e-10
tf = 3000
#force evaluation at every second
t_pts = np.linspace(t0, tf, tf)  # set number of points between t=t0 and t=tf. OPTIONAL
#my preferred starting configuration
sol_Dhooge =  solve_ivp(poly_Dhooge, (t0, tf), y0, t_eval=t_pts, rtol=relerr, atol=abserr, method='BDF', dense_output=True )  
#sol =  solve_ivp(poly_AKS, (t0, tf), y0, t_eval=t_pts,   rtol=relerr, atol=abserr, method='BDF', dense_output=True )  

In [ ]:
#sol_Dhooge

In [ ]:
#illustrate usage when needed
#Plot_Conversion(sol_AKS, M0, T)
#Plot_Conversion(sol_Dhooge, M0, T)
#Plot_Conversion_Hours(sol, M0, T)

In [ ]:
#Plot_Conversion(sol_AKS, M0, T)

In [ ]:
#illustrate usage when needed
Plot_2_Conversions(sol_AKS, sol_Dhooge, M0, '80')
last_M1 = sol_AKS.y[1,-1]
last_M2 = sol_Dhooge.y[1,-1]
print(last_M1)
print(last_M2)

In [ ]:
#compare moments graphicaly
def Plot_lambdas(ode_result1, ode_result2):
    #Create a figure area
    fig = plt.figure(figsize=(12,6))
    # Create two plots
    #fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    # Plot the first set of data
    # plt. gca()
#  [I0, M0, lambda0, lambda1, lambda2, mu0, mu1, mu2, I2]  [Dhooge is result2]
    ax1.plot(ode_result2.t, ode_result2.y[8,:], label='lam0', )
    ax2.plot(ode_result2.t, ode_result2.y[3,:], label='lam1', )
    ax2.plot(ode_result2.t, ode_result2.y[4,:], label='lam2', )
    
    ax1.set_ylabel('Concentration')
    ax1.set_xlabel('Time(s)')
    ax1.grid()
    ax1.set_title(f'Run Temperature = {T} ' )
    ax1.legend() #NOTE essential for the labels
    

In [ ]:
#Plot_lambdas(sol_AKS, sol_Dhooge)

In [ ]:
Plot_Number_and_Weight_Averages(sol_AKS, T)


In [ ]:
#Xn = M1 / M0
#y0 = [I0, M0, lambda0, lambda1, lambda2, mu0, mu1, mu2] AKS
print( sol_AKS.y[3][-1]  /sol_AKS.y[2][-1]) #matches Predici  #lambda1/ lambda0
#Xw = M1^2 / M0
print(sol_AKS.y[3][-1] * sol_AKS.y[3][-1] /sol_AKS.y[2][-1]) #lambda1**2/Lambda0
#Xz = M2 / M1
print( sol_AKS.y[4][-1]  /sol_AKS.y[3][-1]) #matches Predici # Lambda2/ Lambda1

In [ ]:
#Xn = M1 / M0
#y0 = [I0, M0, lambda0, lambda1, lambda2, mu0, mu1, mu2] AKS
print( sol_Dhooge.y[6][-1]  /sol_Dhooge.y[5][-1]) #mu1/mu0
#Xw = M1^2 / M0
print(sol_Dhooge.y[6][-1] * sol_Dhooge.y[6][-1] /sol_Dhooge.y[5][-1]) # (mu1**2/M0)
#Xz = M2 / M1
print( sol_Dhooge.y[7][-1]  /sol_Dhooge.y[6][-1]) #should be 163 GOT IT #HIS Mu2 / Mu1 
 

In [ ]:
#switch Dhooge moments
#Xn = M1 / M0
#y0 = [I0, M0, lambda0, lambda1, lambda2, mu0, mu1, mu2] AKS
print( sol_Dhooge.y[3][-1]  /sol_Dhooge.y[2][-1]) #lam1/lam0
#Xw = M1^2 / M0
print(sol_Dhooge.y[3][-1] * sol_Dhooge.y[3][-1] /sol_Dhooge.y[2][-1]) # (lam1**2/lam0)
#Xz = M2 / M1
print( sol_Dhooge.y[4][-1]  /sol_Dhooge.y[3][-1])  #HIS Lam2 / Lam1


In [ ]:
def Plot_Conversions_Versus_Chain_Length(ode_result1,ode_result2 , M0, T):
    conversion =1 - (ode_result1.y[1][1:]/M0 )
    #Lambda1 / Lambda 0
    Xn_AKS = ode_result1.y[3][1:]/ode_result1.y[2][1:]  #avoid divide by zero at time 0
    Xn_Dhooge =  ode_result2.y[6][1:]/ode_result2.y[5][1:]  #avoid divide by zero at time 0
    plt.plot( conversion, Xn_AKS,  label='AKS'  )
    plt.plot( conversion, Xn_Dhooge,  label='Dhooge'  )
    plt.ylabel ('Chain Length' )
    plt.xlabel('Conversion')
    plt.title( 'Conversion versus Chain Length')
    plt.legend()
    plt.grid()    
    plt.show()
    return


In [ ]:
Plot_Conversions_Versus_Chain_Length(sol_AKS,sol_Dhooge, M0, T)


In [ ]:
Plot_All_Moments(sol_AKS, 'AKS')

In [ ]:
Plot_All_Moments(sol_Dhooge, 'Dhooge')